# Importing libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import random as rn
from sklearn.preprocessing import  StandardScaler

In [ ]:
import tensorflow as tf

# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten , Input ,BatchNormalization, Bidirectional, Layer , Maximum , Softmax
from tensorflow.keras.layers import  GlobalAveragePooling2D, AveragePooling2D , Conv1D , MaxPooling1D, Average, Dropout, LayerNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, GRU, ReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,balanced_accuracy_score
import joblib

# Data Pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cam = '1'

image = '/content/drive/MyDrive/Colab Notebooks/raw/image_1.npy'       
name = '/content/drive/MyDrive/Colab Notebooks/raw/name_1.npy'
label = '/content/drive/MyDrive/Colab Notebooks/raw/label_1.npy'

img_1 = np.load(image)
label_1 = np.load(label)
label_1 = np.reshape(label_1, (label_1.shape[0], ))
name_1 = np.load(name)

cam = '2'

image = '/content/drive/MyDrive/Colab Notebooks/raw/image_2.npy'       
name = '/content/drive/MyDrive/Colab Notebooks/raw/name_2.npy'
label = '/content/drive/MyDrive/Colab Notebooks/raw/label_2.npy'

img_2 = np.load(image)
label_2 = np.load(label)
label_2 = np.reshape(label_2, (label_2.shape[0], ))
name_2 = np.load(name)

In [ ]:
print(img_1.shape)
print(name_1.shape)
print(img_2.shape)
print(name_2.shape)

(294677, 32, 32)
(294677,)
(294677, 32, 32)
(294677,)


In [ ]:
#Visualizing data

class_name = [    '?????' ,
                'Falling hands' ,
                 'Falling knees' ,
                 'Falling backwards',
                 'Falling sideward' ,
                ' Falling chair',
                ' Walking',
                 'Standing' ,
                 'Sitting' ,
                 'Picking object' ,
                 'Jumping' ,
                 'Laying' ]

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img_1[i], cmap = 'gray')
    plt.xlabel(class_name[label_1[i]])
plt.show()

In [ ]:
X_train_1,X_rem_1,y_train_1,y_rem_1 = train_test_split(img_1,label_1,
                                                         train_size = 0.5,
                                                         random_state = 42,
                                                         )

X_val_1,X_test_1,y_val_1,y_test_1 = train_test_split(X_rem_1,y_rem_1,
                                                         test_size = 0.5,
                                                         random_state = 42,
                                                         )
print('*'*20)
print('X_train_1 shape : ',X_train_1.shape)
print('X_test_1 shape : ',X_test_1.shape)
print('X_val_1 shape : ',X_val_1.shape)
print('y_train_1 shape : ',y_train_1.shape)
print('y_test_1 shape : ',y_test_1.shape)
print('y_val_1 shape : ',y_val_1.shape)



X_train_2,X_rem_2,y_train_2,y_rem_2 = train_test_split(img_2,label_2,
                                                         train_size = 0.5,
                                                         random_state = 42,
                                                         )

X_val_2,X_test_2,y_val_2,y_test_2 = train_test_split(X_rem_2,y_rem_2,
                                                         test_size = 0.5,
                                                         random_state = 42,
                                                         )

print('*'*20)
print('X_train_2 shape : ',X_train_2.shape)
print('X_test_2 shape : ',X_test_2.shape)
print('X_val_2 shape : ',X_val_2.shape)
print('y_train_2 shape : ',y_train_2.shape)
print('y_test_2 shape : ',y_test_2.shape)
print('y_val_2 shape : ',y_val_2.shape)

********************
X_train_1 shape :  (147338, 32, 32)
X_test_1 shape :  (73670, 32, 32)
X_val_1 shape :  (73669, 32, 32)
y_train_1 shape :  (147338,)
y_test_1 shape :  (73670,)
y_val_1 shape :  (73669,)
********************
X_train_2 shape :  (147338, 32, 32)
X_test_2 shape :  (73670, 32, 32)
X_val_2 shape :  (73669, 32, 32)
y_train_2 shape :  (147338,)
y_test_2 shape :  (73670,)
y_val_2 shape :  (73669,)


In [ ]:
def convert_to_categorical (y_train, y_test, y_val) :
    print('--------------------------To Categorical ---------------------------')
    Y_train = to_categorical(y_train, 21)
    Y_test = to_categorical(y_test, 21)
    Y_val = to_categorical(y_val,21)
    
    print('Before    : ', y_train[0])
    print('After     : ',Y_train[0])
    return Y_train, Y_test, Y_val 

print(y_train_1[0])
Y_train_1 , Y_test_1 , Y_val_1 = convert_to_categorical (y_train_1, y_test_1, y_val_1)
Y_train_2 , Y_test_2 , Y_val_2 = convert_to_categorical (y_train_2, y_test_2, y_val_2)

10
--------------------------To Categorical ---------------------------
Before    :  10
After     :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------------------------To Categorical ---------------------------
Before    :  10
After     :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
X_train_1_scaled = X_train_1/255.0
X_train_1_scaled = X_train_1_scaled.reshape(X_train_1_scaled.shape[0], X_train_1_scaled.shape[1]*X_train_1_scaled.shape[1])
X_train_1_scaled = np.expand_dims(X_train_1_scaled, axis=-1)
X_train_2_scaled = X_train_2/255.0
X_train_2_scaled = X_train_2_scaled.reshape(X_train_2_scaled.shape[0], X_train_2_scaled.shape[1]*X_train_2_scaled.shape[1])
X_train_2_scaled = np.expand_dims(X_train_2_scaled, axis=-1)

X_val_1_scaled = X_val_1/255.0
X_val_1_scaled = X_val_1_scaled.reshape(X_val_1_scaled.shape[0], X_val_1_scaled.shape[1]*X_val_1_scaled.shape[1])
X_val_1_scaled = np.expand_dims(X_val_1_scaled, axis=-1)
X_val_2_scaled = X_val_2/255.0
X_val_2_scaled = X_val_2_scaled.reshape(X_val_2_scaled.shape[0], X_val_2_scaled.shape[1]*X_val_2_scaled.shape[1])
X_val_2_scaled = np.expand_dims(X_val_2_scaled, axis=-1)

X_test_1_scaled = X_test_1/255.0
X_test_1_scaled = X_test_1_scaled.reshape(X_test_1_scaled.shape[0], X_test_1_scaled.shape[1]*X_test_1_scaled.shape[1])
X_test_1_scaled = np.expand_dims(X_test_1_scaled, axis=-1)
X_test_2_scaled = X_test_2/255.0
X_test_2_scaled = X_test_2_scaled.reshape(X_test_2_scaled.shape[0], X_test_2_scaled.shape[1]*X_test_2_scaled.shape[1])
X_test_2_scaled = np.expand_dims(X_test_2_scaled, axis=-1)

print(X_train_1_scaled.shape)
print(X_test_1_scaled.shape)
print(X_val_1_scaled.shape)

print(X_train_2_scaled.shape)
print(X_test_2_scaled.shape)
print(X_val_2_scaled.shape)

(147338, 1024, 1)
(73670, 1024, 1)
(73669, 1024, 1)
(147338, 1024, 1)
(73670, 1024, 1)
(73669, 1024, 1)


In [ ]:
Y_train_1 = np.expand_dims(y_train_1, axis=-1)
Y_test_1 = np.expand_dims(y_test_1, axis=-1)
Y_val_1 = np.expand_dims(y_val_1, axis=-1)
print(Y_val_1[1])

[8]


# LSTM

In [ ]:
def display_result(y_test , y_pred) :
    print('Accuracy score : ', accuracy_score(y_test , y_pred) )

def model_Expert( shape ) :     

    inputs = Input(shape = (shape, 1))
    layer_1 = Bidirectional(LSTM(32, return_sequences=True))(inputs)
    layer_2 = Bidirectional(LSTM(32))(layer_1)
    layer_3 = Dropout(rate = 0.25)(layer_2)
    # ----------------------------
    layer_4 = Flatten()(layer_3)
    layer_5 = Dense(units =  128,
                    activation = tf.nn.relu)(layer_4)
    layer_6 = Dropout(rate = 0.25)(layer_5)
    #-----------------------------         
    layer_7 = Dense(units =  64,
                    activation = tf.nn.relu)(layer_6)
    y_hat = Dense(units = 21,
                    activation = 'softmax')(layer_7)
    
    model = Model(inputs= inputs, outputs=y_hat)

    return model 

In [ ]:
expert_1 = model_Expert( 1024 )
expert_1.summary()
expert_1.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=["acc"]
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 1024, 64)         8704      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8320  

In [ ]:
print(Y_val_1.shape)
print(X_val_1_scaled.shape)

(73669, 1)
(73669, 1024, 1)


In [ ]:

"""
ONLY REQUIRED FOR TRAINING. SKIP THIS CELL FOR PREDICTION.
"""

f1_callback_concat = ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/Fall prediction/Weights/expert_lstm_1.hdf5", 
                              monitor='val_f1_score', 
                              verbose=1, 
                              mode='max')
history_1 = expert_1.fit(x = X_val_1_scaled , 
                      y=  Y_val_1,           
                      epochs= 20, 
                      batch_size = 50 , 
                      callbacks = [f1_callback_concat]
                     )


Epoch 1/20
1474/1474 [==============================] - ETA: 0s - loss: 0.9964 - acc: 0.6085
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/Fall prediction/Weights/expert_lstm_1.hdf5
1474/1474 [==============================] - 2095s 1s/step - loss: 0.9964 - acc: 0.6085
Epoch 2/20
1474/1474 [==============================] - ETA: 0s - loss: 0.8391 - acc: 0.6788
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/Fall prediction/Weights/expert_lstm_1.hdf5
1474/1474 [==============================] - 2087s 1s/step - loss: 0.8391 - acc: 0.6788
Epoch 3/20
1474/1474 [==============================] - ETA: 0s - loss: 0.7868 - acc: 0.7065
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/Fall prediction/Weights/expert_lstm_1.hdf5
1474/1474 [==============================] - 2087s 1s/step - loss: 0.7868 - acc: 0.7065
Epoch 4/20
1474/1474 [==============================] - ETA: 0s - loss: 0.6778 - acc: 0.7778
Epoch 4: saving model to /content/drive/MyD

In [ ]:
expert_1.load_weights("/content/drive/MyDrive/Colab Notebooks/Fall prediction/Weights/expert_lstm_1.hdf5")
# print(expert_1.evaluate(X_val_1_scaled , Y_val_1))
# print(expert_1.evaluate(X_test_1_scaled , Y_test_1))


y_val = np.argmax(expert_1.predict(X_val_1_scaled ), axis = 1 )
display_result(y_val_1, y_val)

print('*'*20)
y_test = np.argmax(expert_1.predict(X_test_1_scaled ), axis = 1 )
display_result(y_test_1, y_test)

2303/2303 [==============================] - 646s 280ms/step
Accuracy score :  0.9026727660209858
********************
2303/2303 [==============================] - 631s 274ms/step
Accuracy score :  0.9005158137640831
